# 3-Color 3D TrackMate + Napari Visualization (v 0.4.0)

This is code to analyze 3-color translation movies that have a corresponding track file. The red channel is assumed to be the one corresponding to the tracks.

### Initiation

In [ ]:
# To manipulate arrays
import numpy as np 
# To import images 

# To handle track DataFrames
import pandas as pd

# To import images
from skimage import io 
from skimage.io import imread
from skimage import exposure, img_as_uint, img_as_float

# To make plots
import matplotlib as mpl 
import matplotlib.pyplot as plt 

# To work inline; change to %matplotlib notebook for interactive plotting
%matplotlib inline 

# Napari 
%gui qt5 
from skimage import data
import napari

# To create interactive elements
import ipywidgets as widgets 
from ipywidgets import interact, interactive, fixed, interact_manual, Button, HBox, VBox, Layout, GridspecLayout
from ipywidgets.embed import embed_minimal_html, dependency_state

# Image processing and filters
from skimage.filters import difference_of_gaussians

# Iteration tools such as groupby 
import itertools

# For directories 
import os

### Set current working directory, filenames,  pixel dimensions, and crop size

In [ ]:
# Set the current working directory
os.chdir('C:/Users/tim_s/OneDrive - Colostate/Stasevich Lab/Lab Management/Dry Lab/Python/Python Scripts/LabScripts/TestData/')

Set directory of 3D video files and tracking files (must be same):

In [ ]:
video_directory = 'C:/Users/tim_s/OneDrive - Colostate/Stasevich Lab/Lab Management/Dry Lab/Python/Python Scripts/LabScripts/TestData/'

Set filename for the video file. Track csv filename must be 'spots in tracks statistics' + filename:

In [ ]:
video_3D_filename = 'Hela_confocal.tif'
#video_3D_filename = '200827_Cont_1_cRNA_cFlag_cGFP.tif'
#video_3D_filename = '200827_CA24h_2_cRna_cFlag_cGFP.tiff'
video_3D_tracks_filename = 'Spots in tracks statistics_' + video_3D_filename[:-4] + '.csv' 
video_3D_file_and_path = video_directory + video_3D_filename 
video_3D_tracks_file_and_path = video_directory +  video_3D_tracks_filename 

Set the size of the crops you want to use in track array:

In [ ]:
crop_pad = 5; # size of crops...produces crops of dimensions 2*crop_pad + 1
crop_dim = 2*crop_pad + 1

Specify X, Y, and Z dimensions of voxels in track array crops or 3D videos:

In [ ]:
xy_pixel_size, z_pixel_size = 130, 500
z_renorm = z_pixel_size/xy_pixel_size

### Load in track csv file into a dataframe

In [ ]:
trackMate = pd.read_csv(video_3D_tracks_file_and_path)
trackMate

In [ ]:
# Renormalize the z-position, so tracks appears with proper z-dimension in Napari 
trackMate['POSITION_Z_Renorm'] = z_renorm*trackMate['POSITION_Z']

In [ ]:
# Useful code for sorting tracks by their length...however, incompatible with Napari
# g = trackMate.groupby('TRACK_ID')
# sg = sorted(g,  # iterates pairs of (key, corresponding subDataFrame)
#                 key=lambda x: len(x[1]),  # sort by number of rows (len of subDataFrame)
#                 reverse=True)  # reverse the sort i.e. largest first
# trackMate=pd.concat([group for name, group in sg])

In [ ]:
# Find the TRACK_ID for every unique track
myTrackNs=trackMate.TRACK_ID.unique()
n_tracks=myTrackNs.size

In [ ]:
# Create new column in track array with XY positions adjusted for crops (basically make XY middle of crop)
trackMate['POSITION_X_CROP']=0*trackMate['POSITION_X']+crop_pad+0.5
trackMate['POSITION_Y_CROP']=0*trackMate['POSITION_Y']+crop_pad+0.5

### Load in 3D video to make track array (tarpy) 

In [ ]:
# Loading video as a numpy array
video_3D = imread(video_3D_filename)
#video_3D = imread('200827_Cont_1_cRNA_cFlag_cGFP.tif')
#video_3D = imread('200827_CA24h_2_cRna_cFlag_cGFP.tiff')
#video_3D = imread('E:/Tim_tracking videos_all zs/TA02_90m.tif')
n_frames = video_3D.shape[0]
z_slices = video_3D.shape[1]
height_y = video_3D.shape[2]
width_x = video_3D.shape[3]
n_channels = video_3D.shape[4]
video_3D.shape

In [ ]:
# Find nice intensity ranges for display later; tracks assumed to be red channel
allr = np.ma.masked_equal(video_3D[:,:,:,:,0],0).compressed().flatten() # Drop zeros
allg = np.ma.masked_equal(video_3D[:,:,:,:,1],0).compressed().flatten() # Drop zeros
allb = np.ma.masked_equal(video_3D[:,:,:,:,2],0).compressed().flatten() # Drop zeros
r_range = [np.mean(trackMate['MIN_INTENSITY'])/1.5,1.5*np.mean(trackMate['MAX_INTENSITY'])]
g_range = [np.median(allg)-0.1*np.std(allg),np.median(allg)+7*np.std(allg)]
b_range = [np.median(allb)-0.1*np.std(allb),np.median(allb)+7*np.std(allb)]
range = (np.min([r_range[0], g_range[0], b_range[0]]), np.max([r_range[1], g_range[1], b_range[1]]))
n, bins, patches = plt.hist(allr, 100,range,edgecolor='r',histtype='step')
n, bins, patches = plt.hist(allg, 100, range,edgecolor='g', histtype='step')
n, bins, patches = plt.hist(allb, 100, range,edgecolor='b',histtype='step')
plt.xlabel('Background Subtracted Intensity')
plt.ylabel('Number')
plt.title('Histogram of Intensities')
plt.grid(True)
#plt.xlim(0, np.max([r_range[1], g_range[1], b_range[1]]))
plt.show()

### Visualize tracks in original 4D image with Napari (optional)

In [ ]:
# Set up tracks in Napari format for viewing
trackMate_napari=trackMate[['TRACK_ID','POSITION_T','POSITION_Z_Renorm','POSITION_Y','POSITION_X']]

In [ ]:
# ##Comment/uncomment this using Ctrl + 'a' to select all and then Ctrl + 'l' to uncomment/comment
# # View in Napari using tracks layer:
# viewer = napari.Viewer()
# viewer.add_image(video_3D[:, :, :, :, 0], colormap='red',
#                  name='red',blending="additive", scale=[1, z_renorm, 1, 1],
#                  contrast_limits=r_range)
# viewer.add_image(video_3D[:, :, :, :, 1], colormap='green',
#                  name='green',blending="additive", scale=[1, z_renorm, 1, 1],
#                  contrast_limits=g_range)
# viewer.add_image(video_3D[:, :, :, :, 2], colormap='blue',
#                  name='blue',blending="additive", scale=[1, z_renorm, 1, 1],
#                  contrast_limits=b_range)
# viewer.add_tracks(trackMate_napari.values, tail_width = 3, tail_length=25, name="my_tracks")

### Making and exploring an array of 4D track crops using Napari 

In [ ]:
# This is an empty array that will hold ALL the crops of each track
myCropsAll=np.zeros((n_tracks,n_frames,z_slices,2*crop_pad+1,2*crop_pad+1,n_channels))

In [ ]:
# Assign each crop to empty array defined above
myi=0
for myN in myTrackNs:
    myTrack = trackMate[(trackMate['TRACK_ID'] == myN) & (trackMate['POSITION_X']<width_x-crop_pad-1) & (trackMate['POSITION_X']>crop_pad+1) & (trackMate['POSITION_Y']<height_y-crop_pad-1) & (trackMate['POSITION_Y']>crop_pad+1) ]
    myTimes=myTrack['POSITION_T'].values.astype(int) 
    myX=myTrack['POSITION_X'].round(0).values.astype(int)
    myY=myTrack['POSITION_Y'].round(0).values.astype(int)
    myZ=myTrack['POSITION_Z'].round(0).values.astype(int)
    myTrack_napari = myTrack[['TRACK_ID','POSITION_T','POSITION_Z_Renorm','POSITION_Y_CROP','POSITION_X_CROP']]
    tind = 0
    for t in myTimes:
        myCropsAll[myi,t,:,:,:,:] = video_3D[t,:,myY[tind]-crop_pad:myY[tind]+crop_pad+1,myX[tind]-crop_pad:myX[tind]+crop_pad+1,:]
        tind = tind + 1
    myi = myi+1

In [ ]:
# # View in Napari; track layer doesn't work in this format
# viewer = napari.Viewer()
# viewer.add_image(myCropsAll[:, :, :, :, :, 0], colormap='red',
#                  name='red',blending="additive", scale=[1, 1, z_renorm, 1, 1],
#                  contrast_limits=r_range)
# viewer.add_image(myCropsAll[:, :, :, :, :, 1], colormap='green',
#                  name='green',blending="additive", scale=[1, 1, z_renorm, 1, 1],
#                  contrast_limits=g_range)
# viewer.add_image(myCropsAll[:, :, :, :, :, 2], colormap='blue',
#                  name='blue',blending="additive", scale=[1,1, z_renorm, 1, 1],
#                  contrast_limits=b_range)

### Saving track array (tarpy) as video_3D_tracks_filename + crop_pad + '.tif'

In [ ]:
# Save our multicolour stack to a new ImageJ-compatible TIF file
# tifffile wants our array to be in TZCYXS order for imageJ compatibility
myCropsAll4D = np.hstack(myCropsAll.swapaxes(2,4)).swapaxes(1,3)
myCropsAll4D_Z = np.hstack(myCropsAll4D).swapaxes(1,2)
my4DArray = np.moveaxis(myCropsAll4D_Z.astype(np.int16),-1,1) 
io.imsave(video_directory + video_3D_tracks_filename[:-4] + '_crop_pad_' + str(crop_pad) + '.tif',
        my4DArray,  # so move C from the end to second dimension
        imagej=True,
        resolution=(1/xy_pixel_size,1/xy_pixel_size),  # store x and y resolution in pixels/nm
        metadata={'spacing':z_pixel_size,'unit':'nm'})  # store z spaxing in nm and set units to nm

### Reading in track array (tarpy)  

In [ ]:
# # Quick check that there is no loss of data when reading out and reading back in
video_tarpy_filename = video_3D_tracks_filename[:-4] + '_crop_pad_' + str(crop_pad) + '.tif'
video_tarpy_filename_and_path = video_directory + video_tarpy_filename
video_tarpy = imread(video_tarpy_filename_and_path)

In [ ]:
s1 = video_tarpy_filename.find('crop_pad_')
s2 = video_tarpy_filename.find('.tif')
crop_pad = int(video_tarpy_filename[s1+9:s2])
crop_dim = 2*crop_pad+1
n_frames = int(video_tarpy.shape[2]/crop_dim)
z_slices = int(video_tarpy.shape[0])
n_channels = int(video_tarpy.shape[3])
n_tracks = int(video_tarpy.shape[1]/crop_dim)
[video_tarpy.shape, n_frames, z_slices, n_channels, n_tracks, crop_dim, z_renorm]

In [ ]:
# Recreate myCropsAll array from tarpy
myCropsAll = np.zeros((n_tracks,n_frames,z_slices,crop_dim,crop_dim,n_channels))
for n in np.arange(n_tracks):
    for t in np.arange(n_frames):
        myCropsAll[n,t] = video_tarpy[:,n*crop_dim:n*crop_dim+crop_dim,t*crop_dim:t*crop_dim+crop_dim,:]
myCropsAll = np.swapaxes(myCropsAll,3,4)  # Must swap x and y after imread so it matches what we saved

In [ ]:
# # Make a napari track label dataframe to see which track is which during visualization of track array
zeros = np.zeros(n_tracks)
step = 2*crop_pad+1
myLabelTrackXDF=pd.DataFrame(np.array([myTrackNs, zeros, zeros, zeros, np.arange(crop_pad, step*myTrackNs.size, step)]).T,
                            columns=['TRACK_ID', 'POSITION_T', 'POSITION_Z', 'POSITION_Y', 'POSITION_X'])
myLabelTrackYDF=pd.DataFrame(np.array([myTrackNs, zeros, zeros, np.arange(crop_pad, step*myTrackNs.size, step),zeros]).T,
                            columns=['TRACK_ID', 'POSITION_T', 'POSITION_Z', 'POSITION_Y', 'POSITION_X'])

In [ ]:
# Find nice intensity ranges for display later; tracks assumed to be red channel
allr = np.ma.masked_equal(video_tarpy[:,:,:,0],0).compressed().flatten() # Drop zeros
allg = np.ma.masked_equal(video_tarpy[:,:,:,1],0).compressed().flatten() # Drop zeros
allb = np.ma.masked_equal(video_tarpy[:,:,:,2],0).compressed().flatten() # Drop zeros
r_range = [np.mean(trackMate['MIN_INTENSITY'])/1.5,1.5*np.mean(trackMate['MAX_INTENSITY'])]
g_range = [np.median(allr)-3*np.std(allr),np.median(allg)+8*np.std(allg)]
b_range = [np.median(allr)-3*np.std(allr),np.median(allb)+8*np.std(allb)]
range = (np.min([r_range[0], g_range[0], b_range[0]]), np.max([r_range[1], g_range[1], b_range[1]]))
n, bins, patches = plt.hist(allr, 100,range,edgecolor='r',histtype='step')
n, bins, patches = plt.hist(allg, 100, range,edgecolor='g', histtype='step')
n, bins, patches = plt.hist(allb, 100, range,edgecolor='b',histtype='step')
plt.xlabel('Background Subtracted Intensity')
plt.ylabel('Number')
plt.title('Histogram of Intensities')
plt.grid(True)
#plt.xlim(0, np.max([r_range[1], g_range[1], b_range[1]]))
plt.show()

In [ ]:
# # View in Napari; track layer doesn't work in this format
# imgr, imgg, imgb = video_tarpy[:,:,:,0], video_tarpy[:,:,:,1], video_tarpy[:,:,:,2]
# viewer = napari.Viewer()
# viewer.add_image(imgr, colormap='red',
#                  name='red',blending="additive", scale=[z_renorm, 1, 1],
#                  contrast_limits=r_range)
# viewer.add_image(imgg, colormap='green',
#                  name='green',blending="additive", scale=[z_renorm, 1, 1],
#                  contrast_limits=g_range)
# viewer.add_image(imgb, colormap='blue',
#                  name='blue',blending="additive", scale=[z_renorm, 1, 1],
#                  contrast_limits=b_range)
# viewer.add_tracks(myLabelTrackYDF.values, tail_width = 7, tail_length=50, name="labels")

### Functions for making track array 3D masks for analysis and visualization

In [ ]:
# Some functions to make useful 3D masks for quantifying signal intensities

# A cuboid array centered at (cx,cy,cz) with half-lengths (rx,ry,rz) in a volumeXYZ
def myCuboid(cx,cy,cz,rx,ry,rz,volumeX, volumeY, volumeZ):
    x = np.arange(0, volumeX)
    y = np.arange(0, volumeY)
    z = np.arange(0, volumeZ)
    arr = np.zeros((z.size, y.size, x.size))
    stripx = np.heaviside(x[np.newaxis,np.newaxis,:]-(cx-rx),1)-np.heaviside(x[np.newaxis,np.newaxis,:]-(cx+rx+1),1)
    stripy = np.heaviside(y[np.newaxis,:,np.newaxis]-(cy-ry),1)-np.heaviside(y[np.newaxis,:,np.newaxis]-(cy+ry+1),1)
    stripz = np.heaviside(z[:,np.newaxis,np.newaxis]-(cz-rz),1)-np.heaviside(z[:,np.newaxis,np.newaxis]-(cz+rz+1),1)
    mask = stripx*stripy*stripz
    return mask

In [ ]:
# A ellipsoid centered at (cx,cy,cz) with semi-axes of rx, ry, and rz in volumeXYZ
# This is basically the 3D version of the 'disk' in disk-donut quantification
def myCigar(cx,cy,cz,rx,ry,rz,volumeX, volumeY, volumeZ):
    x = np.arange(0, volumeX)
    y = np.arange(0, volumeY)
    z = np.arange(0, volumeZ)
    arr = np.zeros((z.size, y.size, x.size))
    mask = ((1/rx)**2)*(x[np.newaxis,np.newaxis,:]-cx)**2 + ((1/ry)**2)*(y[np.newaxis,:,np.newaxis]-cy)**2 + ((1/rz)**2)*(z[:,np.newaxis,np.newaxis]-cz)**2 < 1
    arr[mask] = 1.
    return arr

In [ ]:
# A capsule that surrounds myCigar(cx,cy,cz,rx,ry,rz,volumeX, volumeY, volumeZ)
# This is basically the 3D version of the 'donut' in 'disk-donut' quantification
def myCapsule(cx,cy,cz,rx,ry,rz,volumeX,volumeY,volumeZ):
    arr1=myCigar(cx,cy,cz,rx+1,ry+1,rz+1,volumeX, volumeY, volumeZ)
    arr2=myCigar(cx,cy,cz,rx+2,ry+2,rz+2,volumeX, volumeY, volumeZ)
    return arr2-arr1

In [ ]:
# # Below we test what these look like in 3D. 
#     # The cuboid shows the volume that would correspond to one 3D crop.
#     # The cigar shows the volume in which signal would be quantified
#     # The capsule shows the volume in which background would be quantified
# viewer = napari.Viewer()
# viewer.add_image(myCuboid(10,10,10,5,5,3,20,20,20), colormap='red',
#                  name='red',blending="additive", scale=[z_renorm,1, 1])
# viewer.add_image(myCapsule(10,10,10,4,4,2,20,20,20), colormap='green',
#                  name='green',blending="additive", scale=[z_renorm,1, 1])
# viewer.add_image(myCigar(10,10,10,4,4,2,20,20,20), colormap='blue',
#                  name='blue',blending="additive", scale=[z_renorm,1, 1])

### Making unique 3D masks for every crop in dataset 

In [ ]:
# Make empty arrays to hold all the arrays of various types of masks 
myCigarMasksAll = np.zeros((n_tracks,n_frames,z_slices,2*crop_pad+1,2*crop_pad+1))
myCapsuleMasksAll = np.zeros((n_tracks,n_frames,z_slices,2*crop_pad+1,2*crop_pad+1))
myBestZMasksAll = np.zeros((n_tracks,n_frames,z_slices,2*crop_pad+1,2*crop_pad+1))

# Loop function over all crops to find best-Z plane and make a custom mask
for n in np.arange(n_tracks):
    for t in np.arange(n_frames):
        curCrop = myCropsAll[n,t,:,crop_pad-1:crop_pad+2,crop_pad-1:crop_pad+2,0]
        myCenterZ = np.argmax(np.mean(difference_of_gaussians(curCrop,1,7),axis=(1,2))) # Find Z with the max average intensity in a central 3x3 square after applying bandpass filter
        myCigarMasksAll[n,t] = myCigar(crop_pad,crop_pad,myCenterZ,2,2,2,int(2*crop_pad+1),int(2*crop_pad+1),z_slices)
        myCapsuleMasksAll[n,t] = myCapsule(crop_pad,crop_pad,myCenterZ,2,2,2,int(2*crop_pad+1),int(2*crop_pad+1),z_slices)
        myBestZMasksAll[n,t] = myCuboid(crop_pad,crop_pad,myCenterZ,5,5,1,2*crop_pad+1,2*crop_pad+1,z_slices)
# Make RGB color versions of each mask
myCigarMasksAllRGB = np.array([myCigarMasksAll,myCigarMasksAll,myCigarMasksAll]).transpose(1,2,3,4,5,0)
myCapsuleMasksAllRGB = np.array([myCapsuleMasksAll,myCapsuleMasksAll,myCapsuleMasksAll]).transpose(1,2,3,4,5,0)
myBestZMasksAllRGB = np.array([myBestZMasksAll,myBestZMasksAll,myBestZMasksAll]).transpose(1,2,3,4,5,0)

### Exploring all track crops, masks, and projections with Napari

In [ ]:
# Various projections and visualizations for checking everything
myCropsAll4D = np.hstack(myCropsAll.swapaxes(2,4)).swapaxes(1,3)
myCigarMasksAll4D = np.hstack(myCigarMasksAll.swapaxes(2,4)).swapaxes(1,3)
myBestZMasksAll4D = np.hstack(myBestZMasksAll.swapaxes(2,4)).swapaxes(1,3)
myCropsAll4D_Z = np.hstack(myCropsAll4D).swapaxes(1,2)
myCigarMasksAll4D_Z = np.hstack(myCigarMasksAll4D).swapaxes(1,2)
myBestZMasksAll4D_Z = np.hstack(myBestZMasksAll4D).swapaxes(1,2)
myCropsAll4D_T = np.hstack(myCropsAll4D.swapaxes(0,1))
myCigarMasksAll4D_T = np.hstack(myCigarMasksAll4D.swapaxes(0,1))
myCropsAll4D_SP = np.hstack((myCropsAll*myBestZMasksAllRGB).swapaxes(2,4)).swapaxes(1,3)
myCropsAll3D_SP_Z = np.amax(np.hstack(myCropsAll4D_SP).swapaxes(1,2),0)
myCropsAll3D_SP_Z_MaxZ = np.amax(myCropsAll*myBestZMasksAllRGB,2)
myCropsAll3D_SP_Z_MaxZ_Mask = np.amax(myBestZMasksAllRGB,2)
myCropsAll3D_SP_T=np.amax((myCropsAll4D_T),0) #max T projection 
myCropsAll2D_XT = np.amax(np.amax(myCropsAll4D_SP,1),1)

In [ ]:
# View in Napari
def myNapariViewer(viewChoice = 'N x T Grid of Smart-Z projections'):
    
    if viewChoice == 'N x 1 Row of Crops; Scroll Z & T':
        myR, myG, myB = myCropsAll4D[:,:,:,:,0], myCropsAll4D[:,:,:,:,1], myCropsAll4D[:,:,:,:,2]
        myScale = [1, z_renorm, 1, 1]
        myLayer = myCigarMasksAll4D ## Could also be myBestZMaskAll4D
        myLabel = myLabelTrackXDF.values 
    
    if viewChoice == 'N x T Grid of Crops; Scroll Z':
        myR, myG, myB = myCropsAll4D_Z[:,:,:,0], myCropsAll4D_Z[:,:,:,1], myCropsAll4D_Z[:,:,:,2]
        myScale = [z_renorm,1,1]
        myLayer = myCigarMasksAll4D_Z 
        myLabel = myLabelTrackYDF.values
    
    if viewChoice == 'N x Z Grid of Crops; Scroll T': # easy to check masks!
        myR, myG, myB = myCropsAll4D_T[:,:,:,0], myCropsAll4D_T[:,:,:,1], myCropsAll4D_T[:,:,:,2]
        myScale = [1,1,1]
        myLayer = myCigarMasksAll4D_T ## Could also be myBestZMaskAll4D
        myLabel = myLabelTrackXDF.values
    
    if viewChoice == 'N x T Grid of Smart-Z projections':  # Best visualization
        myR, myG, myB = myCropsAll3D_SP_Z[:,:,0], myCropsAll3D_SP_Z[:,:,1], myCropsAll3D_SP_Z[:,:,2]
        myScale = [1,1]
        myLayer = myCigarMasksAll4D_Z  
        myLabel = myLabelTrackYDF.values
    
    if viewChoice == 'X x Y x T image; Scroll tracks':  # Cool visualization; High-content
        myR, myG, myB = myCropsAll3D_SP_Z_MaxZ[:,:,:,:,0], myCropsAll3D_SP_Z_MaxZ[:,:,:,:,1], myCropsAll3D_SP_Z_MaxZ[:,:,:,:,2]
        myScale = [1,1,1,1]
        myLayer = myCropsAll3D_SP_Z_MaxZ_Mask
        myLabel=np.array([[0,0,0,0,0]])  # Dummy array...don't really need label for this one

    if viewChoice == 'N x Z grid of max T projection':
        myR, myG, myB = myCropsAll3D_SP_T[:,:,0], myCropsAll3D_SP_T[:,:,1], myCropsAll3D_SP_T[:,:,2]
        myScale = [1,1]
        myLayer = myCigarMasksAll4D_T 
        myLabel = myLabelTrackXDF.values

    if viewChoice == 'N x 1 grid of max XT images':
        myR, myG, myB = myCropsAll2D_XT[:,:,0], myCropsAll2D_XT[:,:,1], myCropsAll2D_XT[:,:,2]
        myScale = [1,1]
        myLayer = myCigarMasksAll4D_T 
        myLabel = myLabelTrackXDF.values

    # Napari Visualization: 
    viewer = napari.Viewer()
    viewer.add_image(myR, colormap='red',
                 name='red',blending="additive", scale=myScale,
                 contrast_limits=r_range)
    viewer.add_image(myG, colormap='green',
                 name='green',blending="additive", scale=myScale,
                 contrast_limits=b_range)
    viewer.add_image(myB, colormap='blue',
                 name='blue',blending="additive", scale=myScale,
                 contrast_limits=g_range)
    viewer.add_image(myLayer, colormap='gray',opacity=0.25,
                 name='layer',blending="additive", scale=myScale)
    viewer.add_tracks(myLabel, tail_width = 7, tail_length=50, name="labels")
    
interact(myNapariViewer, viewChoice=['N x 1 Row of Crops; Scroll Z & T',
                                     'N x T Grid of Crops; Scroll Z',
                                     'N x Z Grid of Crops; Scroll T',
                                     'N x T Grid of Smart-Z projections',
                                     'X x Y x T image; Scroll tracks',
                                     'N x Z grid of max T projection',
                                     'N x 1 grid of max XT images'])

### Data quantification

In [ ]:
#Prepare Data for Cropping using 3D masked crops (can choose)

#Option 1: Cigar-capsule quantification (
mySigMask, myBGMask = myCigarMasksAllRGB ,myCapsuleMasksAllRGB 

#Option 2: Cylinder-ring quantification 
#Need to write this still

mySig=np.mean(np.ma.masked_equal(mySigMask*myCropsAll,0),axis=(2,3,4)) # Mean on Masked crops
myBG=np.mean(np.ma.masked_equal(myBGMask*myCropsAll,0),axis=(2,3,4)) # Mean on Masked crops
myInt=mySig.data-myBG.data  # get unmasked data out of masked array

In [ ]:
# the histogram of the data
allr=np.ma.masked_equal(myInt[:,:,0],0).compressed().flatten() # Drop zeros
allg=np.ma.masked_equal(myInt[:,:,1],0).compressed().flatten() # Drop zeros
allb=np.ma.masked_equal(myInt[:,:,2],0).compressed().flatten() # Drop zeros
n, bins, patches = plt.hist(allr, 50, (-400, 600),edgecolor='r',histtype='step')
n, bins, patches = plt.hist(allg, 50, (-400, 600), edgecolor='g', histtype='step')
n, bins, patches = plt.hist(allb, 50,(-400, 600), edgecolor='b',histtype='step')
plt.xlabel('Background Subtracted Intensity')
plt.ylabel('Number')
plt.title('Histogram of Intensities')
plt.grid(True)
plt.xlim(-400, 600)
plt.show()

In [ ]:
def myIntPlotter(myTOI):
    t = np.arange(n_frames)
    r = np.ma.masked_equal(myInt[myTOI,:,0],0)  # Red channel; mask zeros (skipped frames in tracks)
    g = np.ma.masked_equal(myInt[myTOI,:,1],0) #np.ma.masked_equal(myInt[myTOI,:,1],0)  # Green channel; mask zeros
    b = np.ma.masked_equal(myInt[myTOI,:,2],0) #np.ma.masked_equal(myInt[myTOI,:,2],0)  # Blue channel; mask zeros
    fig, ax = plt.subplots()
    ax.plot(t, r,'ro-',g,'gs-',b,'b^-')
    ax.set(xlabel='frame #', ylabel='Intensity (a.u.)',
       title='Track Intensity vs. time')
    ax.grid()
    plt.show()
    print("TrackMate ID is %d." % myTrackNs[myTOI])
interact(myIntPlotter, myTOI=(0,myInt.shape[0]-1,1))

### Identifying translation spots

In [ ]:
# This will count how many times the green intensity is above myIntThreshhold  
# for a continuous run of length myRunLength. The total length of such continuous
# runs is calculated
myIntThreshhold = 50   # Set this based on intensity histogram above
myRunLength = 4
myID = np.zeros(myInt.shape[0])    # an array to hold the counts

for i in np.arange(myInt.shape[0]):    # going one track at a time
    s=np.where(myInt[i,:,1] > myIntThreshhold, 1, 0) # 1 if > threshhold, 0 otherwise 
    # Below will create a list of continuous runs of 1s (Int>100) and 0s (Int<100)
    full_listing = [(a, list(b)) for a, b in itertools.groupby(s)]
    # Only take the continuous runs of 1s (Int>100)
    all_runs = [b for a, b in full_listing if a == 1]
    # Cacluate the length of each of these runs
    long_run_lengths=[len(a) for a in all_runs if len(a)>=myRunLength]  # could improve?
    # Ouput the sum of the lengths of each continous run
    myID[i] = sum(long_run_lengths)
    
# Now count how many times the runs are longer than myRunLength   
translatingSpots0 = np.where(myID > myRunLength)[0]
translatingSpots = myTrackNs[translatingSpots0]
# Translating spot IDs and the fraction of spots that are translating 
[translatingSpots, translatingSpots.size,translatingSpots.size/myID.size]

In [ ]:
# Checked by eye using Napari visualization above for Hela_confocal.tif
# Maybes by eye: 28, 57, 58,100
translatingSpotsByEye=np.array([  8,   9,  10,  11,  14,  17,  43,  51,  67,  68,  70,  91,  94,
         98, 100, 102, 108, 184, 223, 231, 255, 263, 300])
[translatingSpotsByEye, translatingSpotsByEye.size, translatingSpotsByEye.size/myID.size]

In [ ]:
# Calculate what fraction the algorithm guessed correctly
intersect = [ i for i in translatingSpots if i in translatingSpotsByEye] 
complement = [ i for i in translatingSpots  if i not in translatingSpotsByEye] 
complement2 = [ i for i in translatingSpotsByEye  if i not in translatingSpots] 
np.array([intersect,complement,complement2])

In [ ]:
[len(intersect)/translatingSpotsByEye.size,  # fraction of real spots dectected
len(complement)/translatingSpots.size]       # fraction of spots that might be false positives

In [ ]:
myCropsAll[translatingSpots0].shape

### Checking predicted translating and non-translating tracks by eye

In [ ]:
# Check by eye...algorithm finds many translating spots that check out upon inspection
mySpots=translatingSpots0 # translating indices
#mySpots=np.array([i for i in np.arange(myTrackNs.size) if i not in translatingSpots0])# non-translating indices
zeros = np.zeros(myTrackNs[mySpots].size)
step = 2*crop_pad+1
myLabelTrackYDF2=pd.DataFrame(np.array([myTrackNs[mySpots], zeros, zeros, 
                                    np.arange(crop_pad, step*mySpots.size, step),zeros]).T,
                            columns=['TRACK_ID', 'POSITION_T', 'POSITION_Z', 'POSITION_Y', 'POSITION_X'])
myCropsAll4D_SP = np.hstack((myCropsAll[mySpots]*myBestZMasksAllRGB[mySpots]).swapaxes(2,4)).swapaxes(1,3)
myCropsAll3D_SP_Z = np.amax(np.hstack(myCropsAll4D_SP).swapaxes(1,2),0)
myCigarMasksAll4D_Z = np.hstack(myCigarMasksAll4D).swapaxes(1,2)
myR, myG, myB = myCropsAll3D_SP_Z[:,:,0], myCropsAll3D_SP_Z[:,:,1], myCropsAll3D_SP_Z[:,:,2]
myScale = [1,1]
myLayer = myCigarMasksAll4D_Z  
myLabel = myLabelTrackYDF2
viewer = napari.Viewer()
viewer.add_image(myR, colormap='red',
                 name='red',blending="additive", scale=myScale,
                 contrast_limits=r_range)
viewer.add_image(myG, colormap='green',
                 name='green',blending="additive", scale=myScale,
                 contrast_limits=g_range)
viewer.add_image(myB, colormap='blue',
                 name='blue',blending="additive", scale=myScale,
                 contrast_limits=b_range)
#viewer.add_image(myLayer, colormap='gray',opacity=0.25,
#                 name='layer',blending="additive", scale=myScale)
viewer.add_tracks(myLabel, tail_width = 7, tail_length=50, name="labels")

In [ ]:
#Hela
# Periodicity?: 11, 24, 25, 33, 41, 48, 50, 53, 56, 70, 71, 82

# Control 
# 26% translating according to algorithm
# Local translation! See 43, 119, **184, 342, 527
# false positive: 168, 191, 235, 1255
# need better tracking: 986,1018

# CA24 hrs
# 21% translating according to algorithm
# Seems less obvious translation...
# More false positive